In [92]:
import os
import imp
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import pickle as plk
import importlib.util

###############################################################
# LINUX PATH

#sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
#import lumapi
#sys.path.append(os.path.dirname('/home/trainner/Documentos/Caio/GitReps/Photonics-training/Projects/Microring Resonator/LumericalFiles')) #Current directory

###############################################################
# WINDOWS PATH

spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
lumapi = importlib.util.module_from_spec(spec_win)
os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
sys.path.append(os.path.dirname('C:/Users/Caio/Documents/GitHub/Photonics-training/Projects/Microring Resonator/LumericalFiles/'))
spec_win.loader.exec_module(lumapi)

## Lumapi init

In [93]:
ring = lumapi.FDTD(filename='RINGProject.fsp',hide=False) 

## Functions

In [94]:
def RectInsert(X,XSPAN,Y,YSPAN,Z,ZSPAN,MATERIAL,lumerical):
    
    lumerical.switchtolayout()
    lumerical.addrect()
    lumerical.set({"x":X,"y":Y,"z":Z,
         "x span":XSPAN, "y span":YSPAN, "z span":ZSPAN,
         "material":MATERIAL
        })

## Assembly parameters

In [95]:
StRingLen = 0 # Length of straight sections of the ring
Radius = 3e-6 # Ring radius

StraightSpan = 10e-6 # Length of straight waveguides
Gap = 150e-9 # Gap between waveguides and the ring

BaseHeight = 0.22e-6 # height of the waveguide
BaseWidth = 0.5e-6 # width of the waveguide base

## Assembly

In [96]:
ring.switchtolayout()
ring.selectall()
ring.delete()
ring.addobject("ring_resonator")
ring.set({"x":-StraightSpan/2, "y":0, "z":0,
          "Lc":StRingLen,
          "x span":StraightSpan,
          "radius":Radius,
          "gap":Gap,
          "base height":BaseHeight,
          "base width":BaseWidth})

ring.setnamed("ring resonator::inner_bottom","enabled",0)
ring.setnamed("ring resonator::outer_bottom","enabled",0)
ring.setnamed("ring resonator::arc2","enabled",0)
ring.setnamed("ring resonator::arc3","enabled",0)

RectInsert(-StRingLen-Radius, BaseWidth, -Radius/4, Radius/2, 0, BaseHeight, "Si (Silicon) - Palik", ring)
RectInsert(StRingLen+Radius, BaseWidth, -Radius/4, Radius/2, 0, BaseHeight, "Si (Silicon) - Palik", ring)

ring.addfdtd()
ring.set({"x":0, "y":0.5*(Radius+3*BaseWidth+Gap - Radius/4), "z":0,
          "x span":StraightSpan, "y span":Radius+3*BaseWidth+Gap+Radius/2,
          "background material": "SiO2 (Glass) - Palik",
          "mesh accuracy":3})

ring.addport()
ring.set({"x":-StraightSpan/2.5, "y":Radius+BaseWidth+Gap, "z":0,
          "y span":4*BaseWidth, "z span":2*BaseWidth})
ring.addport()
ring.set({"x":StraightSpan/2.5, "y":Radius+BaseWidth+Gap, "z":0,
          "y span":4*BaseWidth, "z span":2*BaseWidth,
          "direction":'Backward'})
ring.addport()
ring.set({"x":-StRingLen-Radius, "y":-Radius/4, "z":0,
          "injection axis":'y-axis', "x span":4*BaseWidth, "z span":2*BaseWidth})
ring.addport()
ring.set({"x":StRingLen+Radius, "y":-Radius/4, "z":0,
          "injection axis":'y-axis', "x span":4*BaseWidth, "z span":2*BaseWidth})

ring.setglobalsource("wavelength start", 1.5e-6)
ring.setglobalsource("wavelength stop", 1.6e-6)

{!!}



C:\Program Files\Lumerical\v202\api\python\lumapi.py:138: UserWarning: Multiple objects named '::model::rectangle'. Use of this object may give unexpected results.
  warnings.warn(message)


1.0

## Parameter sweeps

### Lc Sweep

In [97]:
# Sweep configs
SPoints = 10       # Sweep points
SRange = [0,3e-6]  # Sweep range

# Sweep creation
ring.deletesweep("Lc_sweep")
ring.addsweep(0)
ring.setsweep("sweep","name","Lc_sweep")
ring.setsweep("Lc_sweep","type","Ranges")
ring.setsweep("Lc_sweep","number of points",SPoints)

LCParameter = {"Name":"LC", "Type":"Length", "Units":"microns",
               "Parameter":"::model::ring resonator::Lc",
               "Start":SRange[0], "Stop":SRange[1]}

Transmission_In = {"Name":"Transmission In",
                   "Result":"::model::FDTD::ports::port 1::T"}
Transmission_Pass = {"Name":"Transmission Pass",
                     "Result":"::model::FDTD::ports::port 2::T"}
Transmission_Drop = {"Name":"Transmission In",
                     "Result":"::model::FDTD::ports::port 4::T"}

ring.addsweepparameter("Lc_sweep",LCParameter)
ring.addsweepresult("Lc_sweep",Transmission_In)
ring.addsweepresult("Lc_sweep",Transmission_Pass)
ring.addsweepresult("Lc_sweep",Transmission_Drop)

'Transmission In1'